In [ ]:
import joblib
import contextlib
from tqdm import tqdm
from joblib import Parallel, delayed

import numpy as np
import itertools
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import math

import time

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback

    try:
        yield tqdm_object

    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [ ]:
TARG_LST = [
    'posterior'
]

ALGO_LST = [
    'LMC',
    'LMCO',
    'aHOLLA'
]

class LangevinSampler:
    def __init__(self, targ, algo, step=0.001, beta=1, d=None):
        assert targ in TARG_LST
        assert algo in ALGO_LST
        
        self.targ = targ
        self.algo = algo
        self.step = step
        self.beta = beta
        self.d = d  # dimension of the parameter


    def _exp(self, theta, x, cache=False):
        z, y = x[:, :-1], x[:, -1]
        if cache:
            if not hasattr(self, '_exp_cache'):
                self._exp_cache = np.exp(np.dot(z, theta))
            return self._exp_cache
        return np.exp(np.dot(z, theta))
    
    def _gradient(self, theta, x, cache=False):
        z, y = x[:, :-1], x[:, -1]
        exp_term = self._exp(theta, x, cache)
        if self.targ == 'posterior':
            return np.sum((1 - y[:, np.newaxis]) * z, axis=0) - np.sum(z / (1 + exp_term)[:, np.newaxis], axis=0) + theta

    def _hessianvectorproduct(self, theta, x, cache=False):
        z, _ = x[:, :-1], x[:, -1]
        grad = self._gradient(theta, x, cache)
        exp_term = self._exp(theta, x, cache)
        if self.targ == 'posterior':
            term = exp_term / (1 + exp_term) ** 2
            hess_inner = term * np.dot(z, grad)   # Shape (N): Inner products <z_i,h>
            hvp_term = np.dot(z.T, hess_inner)    # Shape (d): Sum_{i} <z_i,h>·z_i    
            return grad + hvp_term

    def _vectorlaplacian(self, theta, x, cache=False):
        z, _ = x[:, :-1], x[:, -1]
        exp_term = self._exp(theta, x, cache)
        if self.targ == 'posterior':
            norms_squared = np.linalg.norm(z, axis=1)**2
            term = exp_term * (1 - exp_term) / (1 + exp_term)**3
            return np.sum(z * norms_squared[:, np.newaxis] * term[:, np.newaxis], axis=0)

    def _hessian_term(self, theta, x, cache=False):
        if self.algo == 'LMC':
            return 0
        else:
            return self._hessianvectorproduct(theta, x, cache)

    def _vectorlaplacian_term(self, theta, x, cache=False):
        if self.algo == 'aHOLLA':
            return self._vectorlaplacian(theta, x, cache) / self.beta 
        else:
            return 0

    def _hessiangauproducta(self, theta, x, cache=False):
        z, _ = x[:, :-1], x[:, -1]
        gau_a = np.random.standard_normal(self.d)
        exp_term = self._exp(theta, x, cache)
        if self.targ == 'posterior':
            term = exp_term / (1 + exp_term)**2
            
            hess_inner = term * np.dot(z, gau_a)
            hvp_term = np.dot(z.T, hess_inner) 
            return gau_a - self.step * (gau_a + hvp_term) / 2

    def _hessiangauproductb(self, theta, x, cache=False):
        z, _ = x[:, :-1], x[:, -1]
        gau_b = np.random.standard_normal(self.d)
        exp_term = self._exp(theta, x, cache)
        if self.targ == 'posterior':
            term = exp_term / (1 + exp_term)**2
            
            hess_inner = term * np.dot(z, gau_b)
            hvp_term = np.dot(z.T, hess_inner) 
            return np.sqrt(3) / 6 * self.step * (gau_b + hvp_term)

    
    def _diffusion_term(self, theta, x, cache=False):
        if self.algo == 'LMC':
            return np.random.standard_normal(self.d)
        else:
            return self._hessiangauproducta(theta, x, cache) + self._hessiangauproductb(theta, x, cache)

    
    def sample(self, theta0, x, return_arr=True, runtime=200):
        if runtime is not None:
            n_iter = int(runtime/self.step)
            n_burnin = n_iter
            
        theta = np.ravel(np.array(theta0).reshape(-1))
        
        if return_arr:
            theta_arr = np.zeros((200, self.d))
            
        for n in np.arange(n_iter + n_burnin):
            theta = theta + (- self.step * self._gradient(theta,x,cache=True) + 
                            (self.step ** 2 / 2) * self._hessian_term(theta,x,cache=True) - 
                            (self.step ** 2 / 2) * self._vectorlaplacian_term(theta,x,cache=True) + 
                            np.sqrt(2 * self.step / self.beta) * self._diffusion_term(theta,x,cache=True))
            
            if (n >= (n_iter + n_burnin - 200)) and return_arr:
                index = n - (n_iter + n_burnin - 200)
                theta_arr[index] = theta

            # Clear cache after each iteration
            if hasattr(self, '_exp_cache'):
                del self._exp_cache

        return theta if (not return_arr) else theta_arr

In [ ]:
def generate_data(n, d, theta_true):
    # Generate n d-dimensional standard Gaussian vectors
    #z = np.random.normal(0, 1, (n, d))
    z = np.random.choice([-1, 1], size=(n, d))
    
    # Calculate the parameter p for the Bernoulli random variables
    p = np.exp(np.dot(z, theta_true)) / (1 + np.exp(np.dot(z, theta_true)))

    # Generate y as Bernoulli random variables
    y = np.random.binomial(1, p)

    # Concatenate z and y into one matrix x
    x = np.hstack((z, y.reshape(-1, 1)))  # Reshape y to be a column vector

    return x

In [ ]:
def draw_samples_parallel(sampler, theta0, x, runtime=200, n_chains=50, n_jobs=-1):
    d = len(np.ravel(np.array(theta0).reshape(-1)))
    sampler.d = d

    def _run_single_markov_chain():
        return pd.DataFrame(
            sampler.sample(theta0, x, runtime=runtime),
            columns=[f'component_{i + 1}' for i in range(d)]
        )

    # Create a tqdm progress bar for the number of chains with specific description
    with tqdm(total=n_chains, desc=f"Running {algo} (d={d}, n={n}, step={step})") as pbar:
        with tqdm_joblib(pbar):
            samples_df = Parallel(n_jobs=n_jobs)(
                delayed(_run_single_markov_chain)() for _ in range(n_chains)
            )

    return pd.concat(samples_df, ignore_index=True)

In [ ]:
# Set the parameters
n_values = [100, 200, 500]  # Different numbers of data points
d_values = [5, 10, 20]  # Different values of d
theta0 = None  # Will be defined inside the loop
runtime = 500
n_chains = 50
step_sizes = [0.01]

# Initialize results storage for means and MSE
all_means = {}
all_mse_results = {}
all_running_times = {}

# Run for each value of d
for d in d_values:
    for n in n_values:
        theta_true = np.ones(d)
        x = generate_data(n, d, theta_true)
        
        means = {algo: [] for algo in ALGO_LST}
        mse_results = {algo: [] for algo in ALGO_LST}
        running_times = {algo: [] for algo in ALGO_LST}
        
        # Run for each step size and each algorithm
        for step in step_sizes:
            for algo in ALGO_LST:
                theta0 = 2 * np.ones(d)  # Initialize theta0 for the current d
                sampler = LangevinSampler(targ='posterior', algo=algo, step=step, d=d)
                
                start_time = time.time()
                samples_df = draw_samples_parallel(sampler, theta0, x, runtime=runtime, n_chains=n_chains)
                end_time = time.time()
                
                # Compute mean of the samples for each chain
                mean_samples = samples_df.groupby(samples_df.index // 200).mean().values
                
                # Calculate the MSE for the averaged means
                mse = np.mean(np.linalg.norm((mean_samples - theta_true), axis=1) ** 2)
                mse_results[algo].append(mse)
                means[algo].append(mean_samples)
                running_times[algo].append(end_time - start_time)

        # Store results for this value of d and n
        all_means[(d, n)] = means
        all_mse_results[(d, n)] = mse_results
        all_running_times[(d, n)] = running_times
        
        # Print MSE results for the current value of d
        print(f"\nMSE Results for d={d} and n={n}:")
        mse_df = pd.DataFrame(all_mse_results[(d, n)], index=step_sizes)
        mse_df.index.name = 'Step Size'
        print(mse_df)
        print(running_times)